In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os
data_dir = '/'.join(os.getcwd().split('/')[:-1]) + '/data/'
pop_dir = '/Volumes/mini/data/bg_population_data/'

from mpl_toolkits.basemap import Basemap
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd

from state_lookup import state_lookup

%load_ext autoreload
%autoreload 2

##### USA #### llcrnrlat=20.,llcrnrlon=-135.,urcrnrlat=56.,urcrnrlon=-55.,lat_0=39.5, lon_0=-80,
## Bay Area ##
# state_lookup.get_state_num('California')

In [ ]:
## Sourced from data_retrieval
stations = pd.read_csv(data_dir + 'station_locations.csv')
stations_ = stations.dropna(subset=['latatude'])
stn_zip = zip(stations_['latatude'], stations_['longitude'])

In [ ]:
fig = plt.figure(figsize=(10,20),dpi=200)
map = Basemap(llcrnrlat=37.,llcrnrlon=-123.,urcrnrlat=38.5,urcrnrlon=-121.5,lat_0=37.75, lon_0=-122.5,
             resolution='i', 
    projection='cyl')

map.drawmapboundary(fill_color='aqua')
map.fillcontinents(color='#ddaa66',lake_color='aqua',zorder=1)
map.drawcoastlines()

map.readshapefile(data_dir + 'tl_2016_06_bg/tl_2016_06_bg', '06_bg')
map.scatter(stations['longitude'],stations['latatude'],s=10, color='white',marker='o',zorder=9)
plt.show()

## GEODATABSE Integration
Link: https://www.census.gov/geo/maps-data/data/tiger-data.html (Download Block Group level for a given state)

Documentation: https://www2.census.gov/geo/tiger/TIGER_DP/2016ACS/Metadata/BG_METADATA_2016.txt

In [ ]:
gdf = gpd.read_file(data_dir + 'tl_2016_06_bg/tl_2016_06_bg.shp')
gdf['GEOID'] = gdf['GEOID'].apply(lambda x: '15000US' + x)
gdb_file = gpd.read_file('/Users/ryand/Downloads/ACS_2016_5YR_BG_06_CALIFORNIA.gdb')
del gdb_file['geometry']
geo_db = gdf.merge(gdb_file, on='GEOID')

geo_db['INTPTLON'] = geo_db['INTPTLON'].astype(float)
geo_db['INTPTLAT'] = geo_db['INTPTLAT'].astype(float)

In [ ]:
wking = geo_db[(geo_db['AWATER'] < geo_db['ALAND']) & (geo_db['INTPTLAT'] > 37.25) & (geo_db['INTPTLAT'] < 38.5)
                                            & (geo_db['INTPTLON'] > -123) & (geo_db['INTPTLON'] < -121.5)]

In [ ]:
# B00001e1 == POPULATION
# B00002e1 == HOUSING

In [ ]:
wking['pop_sqm'] = wking['B00002e1']/wking['ALAND']
wking['pop_sqm'] = wking['pop_sqm'].replace(np.inf, 0)

In [ ]:
wking.plot(column='B00001e1',cmap='OrRd',figsize=(20,20))

In [ ]:
def dist(p1, p2):
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def find_closest_station((lat, lon)):
    distance_cache = {}
    for stn in stn_zip:
        distance_cache.update({dist((lat,lon),stn):stn}) 
    min_dist = min(distance_cache.keys())
    return distance_cache[min_dist], min_dist

bg_centers = zip(wking['INTPTLAT'],wking['INTPTLON'])

In [ ]:
wking['closest_stn'], wking['dist'] = zip(*wking[['INTPTLAT','INTPTLON']].apply(find_closest_station, axis=1))

In [ ]:
####### Station Grouping with total population and household

wking[wking['dist'] <= .25].groupby('closest_stn')[['B00001e1','B00002e1']].sum()

In [ ]:
import os

blockpop_filelist = []
blockhome_filelist = []
for a,b,c in os.walk('/Users/ryanduecker/git/public_transit_usability/public_transit_usability/data/tab4/sumfile/prod/2012thru2016/group2'):
    for file in c:
        if file[0] == 'e':
            if file[8:12]=='0001':
                blockpop_filelist.append(file)
            elif file[8:12]=='0002':
                blockhome_filelist.append(file)


df_pop = pd.DataFrame()
for file in blockpop_filelist:
    try:
        df = pd.read_csv('/Users/ryanduecker/git/public_transit_usability/public_transit_usability/data/tab4/sumfile/prod/2012thru2016/group2/' + file)
        df_pop = df_pop.append(df)
    except:
        print(file)

In [ ]:
df = pd.read_csv('population_by_zip_2010.csv')

In [ ]:
zip_pop = df.groupby('zipcode').sum()[['population']]

In [ ]:
zip_pop = zip_pop.reset_index()

In [ ]:
sta = pd.read_csv('station_locations.csv')

In [ ]:
sta['zipcode'] = sta['station'].apply(lambda x: int(x.split(', ')[-1]))

In [ ]:
sta.merge(zip_pop, on='zipcode')

In [ ]:
blocks = pd.read_csv('/Users/ryand/Downloads/All_Geographies/e20171ma0117000.txt')

In [ ]:
import os

blockpop_filelist = []
blockhome_filelist = []
for a,b,c in os.walk('/Users/ryand/Downloads/data/tab4/sumfile/prod/2012thru2016/group2'):
    for file in c:
        if file[0] == 'e':
            if file[8:12]=='0001':
                blockpop_filelist.append(file)
            elif file[8:12]=='0002':
                blockhome_filelist.append(file)


df_pop = pd.DataFrame()
for file in blockpop_filelist:
    try:
        df = pd.read_csv('/Users/ryand/Downloads/data/tab4/sumfile/prod/2012thru2016/group2/' + file)
        df_pop = df_pop.append(df)
    except:
        print(file)

In [ ]:
df_pop